# 🎮 Silksong Gesture Recognition - Complete Training Pipeline

**All-in-One Notebook for Training Gesture Classification Models**

This notebook trains machine learning models to recognize watch gestures for controlling Hollow Knight: Silksong.

---

## 📋 Prerequisites

### 1. Google Colab Setup

**For Colab Pro Users** (Recommended for this project):
- **Runtime**: High-RAM GPU
- **GPU Type**: V100 or A100 for optimal performance
- **Expected Time**: 8-15 min (V100), 5-8 min (A100)

**For Free Tier Users**:
- **Runtime**: GPU (T4)
- **Expected Time**: 20-40 minutes (SVM: 5-10 min, CNN-LSTM: 20-40 min)

**Enable GPU**: Runtime → Change runtime type → Hardware accelerator → GPU → Select GPU type

### 2. Data Organization on Google Drive

Upload your CSV files to Google Drive in this structure:

```
My Drive/
└── silksong_data/
    ├── jump/
    │   ├── jump_sample_01.csv
    │   ├── jump_sample_02.csv
    │   └── ... (30 samples total)
    ├── punch/
    │   ├── punch_sample_01.csv
    │   └── ... (30 samples)
    ├── turn/
    │   ├── turn_sample_01.csv
    │   └── ... (30 samples)
    ├── walk/
    │   ├── walk_sample_01.csv
    │   └── ... (30 samples)
    └── idle/
        ├── idle_sample_01.csv
        └── ... (30 samples)
```

**Total**: 150 CSV files (30 samples × 5 gestures)

### 3. CSV Format

Each CSV should have these columns:
```
accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z, rot_w, rot_x, rot_y, rot_z, sensor, timestamp
```

---

## 🎯 What This Notebook Does

1. **Mount Google Drive** - Access your data
2. **Load & Preprocess Data** - Read all CSV files, extract features
3. **Choose Model Architecture**:
   - **Random Forest (Fast & Robust)**: Ensemble method, great for IMU data
   - **SVM (Traditional)**: Traditional ML with hand-crafted features
   - **CNN-LSTM (Most Accurate)**: Deep learning with temporal awareness
   - **1D CNN (Fast DL)**: Lightweight deep learning option
   - **GRU (Alternative RNN)**: Similar to LSTM but faster
4. **Train & Evaluate** - Train model, show accuracy metrics
5. **Export Model** - Download trained model to use with controller

---

## 1️⃣ Setup & Installation

## ⚙️ CONFIGURATION - Set Your Training Parameters Here

**Customize your training run by editing the values below**

In [ ]:
# ============================================================================
# CONFIGURATION SECTION - Edit these values to customize your training
# ============================================================================

# --- MODEL SELECTION ---
# Choose which model to train. Options:
#   'RANDOM_FOREST' - Fast, robust, great for IMU data (RECOMMENDED for quick iterations)
#   'SVM'           - Traditional ML, good baseline
#   'CNN_LSTM'      - Best accuracy, temporal awareness (RECOMMENDED for production)
#   'CNN_1D'        - Lightweight deep learning, faster than CNN-LSTM
#   'GRU'           - Alternative to LSTM, faster training
#   'ENSEMBLE'      - Combines multiple models (advanced users)

MODEL_TYPE = 'CNN_LSTM'  # ⭐ CHANGE THIS

# --- DATA CONFIGURATION ---
DATA_DIR = "/content/drive/MyDrive/silksong_data/"  # Where your CSV files are stored
GESTURES = ['jump', 'punch', 'turn', 'walk', 'idle']  # Gesture classes to train on

# --- TRAINING HYPERPARAMETERS ---
RANDOM_SEED = 42              # For reproducibility
TEST_SPLIT = 0.2              # 20% of data for testing
VALIDATION_SPLIT = 0.15       # 15% of training data for validation

# Random Forest parameters (if MODEL_TYPE == 'RANDOM_FOREST')
RF_N_ESTIMATORS = 200         # Number of trees (more = better but slower)
RF_MAX_DEPTH = 30             # Tree depth (None for unlimited, 30 is balanced)

# SVM parameters (if MODEL_TYPE == 'SVM')
SVM_KERNEL = 'rbf'            # Kernel type ('rbf', 'linear', 'poly')
SVM_C = 1.0                   # Regularization parameter

# Deep Learning parameters (if MODEL_TYPE in ['CNN_LSTM', 'CNN_1D', 'GRU'])
DL_EPOCHS = 50                # Training epochs (increase for better accuracy)
DL_BATCH_SIZE = 32            # Batch size (32 works well for most cases)
DL_LEARNING_RATE = 0.001      # Learning rate (0.001 is a good default)
DL_DROPOUT = 0.3              # Dropout rate for regularization

# CNN-LSTM specific
WINDOW_SIZE = 50              # Timesteps per window (50 = 1 second at 50Hz)
CNN_FILTERS = (64, 128)       # Filters in Conv layers
LSTM_UNITS = (128, 64)        # Units in LSTM layers

# --- POST-PROCESSING OPTIONS ---
APPLY_BUTTON_SMOOTHING = True      # Smooth button-collected data
APPLY_NOISE_REDUCTION = True       # Remove sensor noise
APPLY_DATA_AUGMENTATION = True     # Augment training data (for deep learning)
AUGMENTATION_FACTOR = 2            # How many augmented samples per original

# --- EXPORT OPTIONS ---
EXPORT_DIR = "/content/drive/MyDrive/silksong_models/"  # Where to save models
SAVE_CONFUSION_MATRIX = True       # Save confusion matrix plot
SAVE_TRAINING_HISTORY = True       # Save training curves (deep learning only)

# ============================================================================
# END OF CONFIGURATION
# ============================================================================

print("✅ Configuration loaded")
print(f"   Model: {MODEL_TYPE}")
print(f"   Gestures: {', '.join(GESTURES)}")
print(f"   Post-processing: Smoothing={APPLY_BUTTON_SMOOTHING}, Noise={APPLY_NOISE_REDUCTION}, Augmentation={APPLY_DATA_AUGMENTATION}")

## 🤖 Model Selection Roundtable - Which Model to Choose?

### Problem Context
**Task**: Real-time gesture recognition for game control  
**Data**: IMU sensors (accelerometer, gyroscope, rotation) at 50Hz  
**Gestures**: 5 classes (jump, punch, turn, walk, idle)  
**Constraints**: <500ms latency, high accuracy on distinct gestures  

### Model Comparison

| Model | Accuracy | Training Time (Colab Pro V100) | Inference Speed | Best For | Pros | Cons |
|-------|----------|-------------------------------|-----------------|----------|------|------|
| **Random Forest** | 85-92% | 2-5 min | 5-10ms | Quick iterations, baseline | ✅ Fast training<br>✅ No GPU needed<br>✅ Robust to noise<br>✅ Feature importance | ❌ Manual features<br>❌ No temporal modeling |
| **SVM (RBF)** | 85-95% | 5-10 min | 10-30ms | Production baseline | ✅ Well-tested<br>✅ Good with small data<br>✅ No GPU needed | ❌ Manual features<br>❌ Slow with large data<br>❌ No temporal info |
| **CNN-LSTM** ⭐ | **92-98%** | 8-15 min | 10-30ms | **Production (best accuracy)** | ✅ **Highest accuracy**<br>✅ Temporal awareness<br>✅ Auto feature learning<br>✅ Handles sequences | ❌ Needs more data<br>❌ GPU required<br>❌ Longer training |
| **1D CNN** | 88-94% | 5-8 min | 5-15ms | Fast deep learning | ✅ Faster than LSTM<br>✅ Auto features<br>✅ GPU accelerated | ❌ Less temporal awareness<br>❌ GPU required |
| **GRU** | 90-96% | 6-10 min | 8-20ms | Alternative to LSTM | ✅ Faster than LSTM<br>✅ Temporal modeling<br>✅ Less parameters | ❌ Slightly less accurate than LSTM<br>❌ GPU required |
| **Ensemble** | **93-99%** | 15-25 min | 20-50ms | Maximum accuracy | ✅ **Best accuracy**<br>✅ Robust predictions | ❌ Slowest<br>❌ Complex deployment<br>❌ High compute |

### 📊 Recommendation for Your Use Case

**🥇 Best Overall: CNN-LSTM**
- **Why**: IMU data is inherently temporal - gestures are sequences of movements. CNN-LSTM excels at this.
- **Accuracy**: 92-98% on similar IMU gesture tasks
- **Latency**: 10-30ms easily meets <500ms requirement
- **With Colab Pro V100**: Train in 8-15 minutes

**🥈 Best for Quick Iterations: Random Forest**
- **Why**: Fast training (2-5 min), no GPU needed, surprisingly good for IMU
- **Use**: Initial experiments, data validation, baseline
- **Accuracy**: 85-92% - good enough for testing

**🥉 Best for Maximum Accuracy: Ensemble (RF + CNN-LSTM + GRU)**
- **Why**: Combines strengths of multiple models
- **Accuracy**: 93-99% but overkill for 5 gestures
- **Trade-off**: Slower inference, complex deployment

### 💡 Practical Workflow

1. **Start**: Train Random Forest → validate data quality (2-5 min)
2. **Iterate**: If RF gets >85%, proceed to CNN-LSTM (8-15 min)
3. **Optimize**: Fine-tune CNN-LSTM hyperparameters (epochs, dropout)
4. **Deploy**: Use CNN-LSTM for production

### 🎯 Why CNN-LSTM Wins for IMU Gesture Recognition

**Temporal Patterns**: Gestures are sequences (e.g., jump = quick upward accel → peak → downward)
- LSTM captures these temporal dependencies
- SVM/RF see each timestep independently

**Automatic Feature Learning**: 
- CNN learns spatial patterns in sensor data
- No need to manually engineer 60+ features

**Similar Research**:
- Human Activity Recognition (HAR): CNN-LSTM achieves 95-98% on smartphone IMU
- Smartwatch gesture recognition: LSTM-based models dominate benchmarks

### 🚀 With Your Colab Pro

**V100 GPU Benefits**:
- CNN-LSTM: 8-15 min (vs 20-40 min on T4)
- Larger batch sizes: 64 or 128 (vs 32 on T4)
- More experiments: Try different architectures quickly

**A100 GPU (if available)**:
- CNN-LSTM: 5-8 min (but overkill for this dataset size)
- Best for: Large-scale experiments, hyperparameter tuning

### ⚡ Bottom Line

**For your 5-gesture smartwatch controller**:
- **Choose CNN-LSTM** - Best accuracy-speed trade-off
- **Use V100 on Colab Pro** - 2-3x faster than free T4
- **Apply data augmentation** - Helps with limited samples

Set `MODEL_TYPE = 'CNN_LSTM'` in the configuration above! ⬆️

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted successfully!")
print("Your data should be at: /content/drive/MyDrive/silksong_data/")

In [ ]:
# Check GPU availability
import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")
print(f"\nGPU devices: {tf.config.list_physical_devices('GPU')}")

if tf.config.list_physical_devices('GPU'):
    print("\n✅ GPU is enabled! Training will be fast (~20-40 min)")
else:
    print("\n⚠️  No GPU detected. Training will be slower (~2-4 hours)")
    print("   To enable: Runtime > Change runtime type > GPU")

In [ ]:
# Install required packages
!pip install -q scikit-learn pandas numpy scipy tensorflow matplotlib seaborn joblib

print("✅ All dependencies installed!")

## 2️⃣ Data Loading & Preprocessing

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import stats
from scipy.fft import fft
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

print("✅ Libraries imported successfully!")

In [ ]:
# Configuration
DATA_DIR = "/content/drive/MyDrive/silksong_data/"
GESTURES = ['jump', 'punch', 'turn', 'walk', 'idle']
RANDOM_SEED = 42

# Verify data directory exists
if not os.path.exists(DATA_DIR):
    print(f"❌ Data directory not found: {DATA_DIR}")
    print("\nPlease create this structure on Google Drive:")
    print("  My Drive/silksong_data/")
    print("    ├── jump/")
    print("    ├── punch/")
    print("    ├── turn/")
    print("    ├── walk/")
    print("    └── idle/")
else:
    print(f"✅ Data directory found: {DATA_DIR}")
    
    # Show what's in the directory
    for gesture in GESTURES:
        gesture_path = os.path.join(DATA_DIR, gesture)
        if os.path.exists(gesture_path):
            count = len([f for f in os.listdir(gesture_path) if f.endswith('.csv')])
            print(f"  {gesture}: {count} CSV files")
        else:
            print(f"  {gesture}: ❌ folder not found")

In [ ]:
def load_gesture_data(data_dir, gestures):
    """
    Load all CSV files for each gesture class.
    
    Returns:
        data: List of (DataFrame, label) tuples
    """
    all_data = []
    
    for gesture_idx, gesture in enumerate(gestures):
        gesture_path = os.path.join(data_dir, gesture)
        
        if not os.path.exists(gesture_path):
            print(f"⚠️  Warning: {gesture} folder not found")
            continue
        
        csv_files = [f for f in os.listdir(gesture_path) if f.endswith('.csv')]
        
        for csv_file in csv_files:
            try:
                df = pd.read_csv(os.path.join(gesture_path, csv_file))
                all_data.append((df, gesture, gesture_idx))
            except Exception as e:
                print(f"❌ Error loading {csv_file}: {e}")
        
        print(f"✅ Loaded {len(csv_files)} samples for '{gesture}'")
    
    return all_data

# Load all data
print("Loading data...\n")
gesture_data = load_gesture_data(DATA_DIR, GESTURES)
print(f"\n✅ Total samples loaded: {len(gesture_data)}")

In [ ]:
# Inspect first sample
if len(gesture_data) > 0:
    sample_df, sample_label, _ = gesture_data[0]
    print(f"Sample gesture: {sample_label}")
    print(f"Shape: {sample_df.shape}")
    print(f"\nColumns: {list(sample_df.columns)}")
    print(f"\nFirst few rows:")
    display(sample_df.head())
else:
    print("❌ No data loaded!")

## 3️⃣ Feature Engineering

Extract time-domain and frequency-domain features from sensor data.

In [ ]:
def extract_features_from_dataframe(df):
    """
    Extract comprehensive features from a single gesture sample.
    
    Features extracted:
    - Time domain: mean, std, min, max, range, median, skew, kurtosis
    - Frequency domain: FFT max, dominant frequency
    - Magnitude features: accel magnitude, gyro magnitude
    
    Returns:
        dict of features
    """
    features = {}
    
    # Separate by sensor type
    accel_data = df[df['sensor'] == 'linear_acceleration']
    gyro_data = df[df['sensor'] == 'gyroscope']
    rot_data = df[df['sensor'] == 'rotation_vector']
    
    # Helper function for time-domain features
    def time_features(series, prefix):
        if len(series) == 0:
            return {}
        return {
            f'{prefix}_mean': np.mean(series),
            f'{prefix}_std': np.std(series),
            f'{prefix}_min': np.min(series),
            f'{prefix}_max': np.max(series),
            f'{prefix}_range': np.max(series) - np.min(series),
            f'{prefix}_median': np.median(series),
            f'{prefix}_skew': stats.skew(series),
            f'{prefix}_kurtosis': stats.kurtosis(series),
        }
    
    # Helper function for frequency features
    def freq_features(series, prefix):
        if len(series) < 4:
            return {f'{prefix}_fft_max': 0, f'{prefix}_dom_freq': 0}
        
        fft_vals = np.abs(fft(series))
        return {
            f'{prefix}_fft_max': np.max(fft_vals[:len(fft_vals)//2]),
            f'{prefix}_dom_freq': np.argmax(fft_vals[:len(fft_vals)//2])
        }
    
    # Accelerometer features
    for axis in ['x', 'y', 'z']:
        col = f'accel_{axis}'
        if col in accel_data.columns:
            series = accel_data[col].dropna()
            features.update(time_features(series, f'accel_{axis}'))
            features.update(freq_features(series, f'accel_{axis}'))
    
    # Gyroscope features
    for axis in ['x', 'y', 'z']:
        col = f'gyro_{axis}'
        if col in gyro_data.columns:
            series = gyro_data[col].dropna()
            features.update(time_features(series, f'gyro_{axis}'))
            features.update(freq_features(series, f'gyro_{axis}'))
    
    # Rotation features (quaternion)
    for axis in ['w', 'x', 'y', 'z']:
        col = f'rot_{axis}'
        if col in rot_data.columns:
            series = rot_data[col].dropna()
            features.update(time_features(series, f'rot_{axis}'))
    
    # Magnitude features
    if len(accel_data) > 0:
        accel_mag = np.sqrt(
            accel_data['accel_x']**2 + 
            accel_data['accel_y']**2 + 
            accel_data['accel_z']**2
        )
        features.update(time_features(accel_mag, 'accel_mag'))
    
    if len(gyro_data) > 0:
        gyro_mag = np.sqrt(
            gyro_data['gyro_x']**2 + 
            gyro_data['gyro_y']**2 + 
            gyro_data['gyro_z']**2
        )
        features.update(time_features(gyro_mag, 'gyro_mag'))
    
    return features

print("✅ Feature extraction function defined")

In [ ]:
# Extract features from all samples
print("Extracting features from all samples...\n")

X_features = []
y_labels = []
y_names = []

for i, (df, gesture_name, gesture_idx) in enumerate(gesture_data):
    try:
        features = extract_features_from_dataframe(df)
        X_features.append(features)
        y_labels.append(gesture_idx)
        y_names.append(gesture_name)
        
        if (i + 1) % 20 == 0:
            print(f"Processed {i + 1}/{len(gesture_data)} samples...")
    except Exception as e:
        print(f"❌ Error extracting features from sample {i}: {e}")

# Convert to DataFrame
X_df = pd.DataFrame(X_features)
y = np.array(y_labels)

# Fill any NaN values with 0
X_df = X_df.fillna(0)

print(f"\n✅ Feature extraction complete!")
print(f"   Features shape: {X_df.shape}")
print(f"   Labels shape: {y.shape}")
print(f"   Feature count: {len(X_df.columns)}")

# Show class distribution
print("\nClass distribution:")
for gesture_idx, gesture in enumerate(GESTURES):
    count = np.sum(y == gesture_idx)
    print(f"  {gesture}: {count} samples")

## 4️⃣ Model Selection

Choose which model architecture to train:

### Option A: SVM (Support Vector Machine)
- **Pros**: Fast training (~5-10 min), good accuracy, works on CPU
- **Cons**: Requires hand-crafted features
- **Use when**: You want quick results or don't have GPU

### Option B: CNN-LSTM (Deep Learning)
- **Pros**: Higher accuracy, learns features automatically
- **Cons**: Slower training (20-40 min with GPU), needs more data
- **Use when**: You have GPU and want best performance

**Set which model to train below:**

In [ ]:
# CHOOSE YOUR MODEL HERE
MODEL_TYPE = "SVM"  # Options: "SVM" or "CNN_LSTM"

print(f"Selected model: {MODEL_TYPE}")

if MODEL_TYPE == "CNN_LSTM" and not tf.config.list_physical_devices('GPU'):
    print("\n⚠️  Warning: CNN-LSTM works best with GPU. Consider enabling GPU or using SVM.")

## 5️⃣ Train/Test Split

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_df, y, 
    test_size=0.2, 
    random_state=RANDOM_SEED,
    stratify=y  # Ensure balanced split
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Data split and scaled!")

## 6️⃣ Model Training - SVM

In [ ]:
if MODEL_TYPE == "SVM":
    print("Training SVM model...\n")
    
    # Create and train SVM
    svm_model = SVC(
        kernel='rbf',
        C=1.0,
        gamma='scale',
        random_state=RANDOM_SEED,
        probability=True  # Enable probability estimates
    )
    
    svm_model.fit(X_train_scaled, y_train)
    
    print("✅ SVM training complete!")
    
    # Evaluate
    train_acc = svm_model.score(X_train_scaled, y_train)
    test_acc = svm_model.score(X_test_scaled, y_test)
    
    print(f"\nTraining accuracy: {train_acc:.2%}")
    print(f"Test accuracy: {test_acc:.2%}")
    
    # Predictions
    y_pred = svm_model.predict(X_test_scaled)
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=GESTURES))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=GESTURES, yticklabels=GESTURES)
    plt.title('Confusion Matrix - SVM')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
else:
    print("Skipping SVM training (CNN_LSTM selected)")

## 7️⃣ Model Training - CNN-LSTM

For CNN-LSTM, we need to reshape data into windows.

In [ ]:
if MODEL_TYPE == "CNN_LSTM":
    print("Preparing data for CNN-LSTM...\n")
    
    def prepare_windowed_data(gesture_data, window_size=50):
        """
        Convert CSV samples into fixed-size windows for CNN-LSTM.
        """
        X_windows = []
        y_windows = []
        
        sensor_cols = ['accel_x', 'accel_y', 'accel_z', 
                       'gyro_x', 'gyro_y', 'gyro_z',
                       'rot_w', 'rot_x', 'rot_y', 'rot_z']
        
        for df, gesture_name, gesture_idx in gesture_data:
            # Process by sensor type
            accel = df[df['sensor'] == 'linear_acceleration'][['timestamp'] + [c for c in ['accel_x', 'accel_y', 'accel_z'] if c in df.columns]]
            gyro = df[df['sensor'] == 'gyroscope'][['timestamp'] + [c for c in ['gyro_x', 'gyro_y', 'gyro_z'] if c in df.columns]]
            rot = df[df['sensor'] == 'rotation_vector'][['timestamp'] + [c for c in ['rot_w', 'rot_x', 'rot_y', 'rot_z'] if c in df.columns]]
            
            # Merge on timestamp
            all_timestamps = pd.DataFrame({'timestamp': sorted(df['timestamp'].unique())})
            merged = all_timestamps.copy()
            
            if len(accel) > 0:
                merged = merged.merge(accel, on='timestamp', how='left')
            if len(gyro) > 0:
                merged = merged.merge(gyro, on='timestamp', how='left')
            if len(rot) > 0:
                merged = merged.merge(rot, on='timestamp', how='left')
            
            # Forward fill and fill remaining with 0
            merged = merged.fillna(method='ffill').fillna(0)
            
            # Extract only sensor columns that exist
            available_cols = [c for c in sensor_cols if c in merged.columns]
            sensor_data = merged[available_cols].values
            
            # Pad or truncate to window_size
            if len(sensor_data) >= window_size:
                # Take middle window
                start = (len(sensor_data) - window_size) // 2
                window = sensor_data[start:start + window_size]
            else:
                # Pad with zeros
                padding = window_size - len(sensor_data)
                window = np.vstack([sensor_data, np.zeros((padding, len(available_cols)))])
            
            # Ensure correct shape
            if window.shape[0] == window_size:
                X_windows.append(window)
                y_windows.append(gesture_idx)
        
        return np.array(X_windows), np.array(y_windows)
    
    # Prepare windowed data
    X_windowed, y_windowed = prepare_windowed_data(gesture_data, window_size=50)
    
    print(f"✅ Windowed data prepared")
    print(f"   Shape: {X_windowed.shape}")
    print(f"   (samples, timesteps, features)")
    
    # Split
    X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(
        X_windowed, y_windowed,
        test_size=0.2,
        random_state=RANDOM_SEED,
        stratify=y_windowed
    )
    
    # Convert labels to categorical
    y_train_cat = to_categorical(y_train_w, num_classes=len(GESTURES))
    y_test_cat = to_categorical(y_test_w, num_classes=len(GESTURES))
    
    print(f"\nTraining set: {X_train_w.shape[0]} samples")
    print(f"Test set: {X_test_w.shape[0]} samples")
    
else:
    print("Skipping CNN-LSTM data preparation (SVM selected)")

In [ ]:
if MODEL_TYPE == "CNN_LSTM":
    print("Building CNN-LSTM model...\n")
    
    # Model architecture
    input_shape = (X_train_w.shape[1], X_train_w.shape[2])  # (timesteps, features)
    
    model = models.Sequential([
        # CNN layers for feature extraction
        layers.Conv1D(64, kernel_size=5, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        
        layers.Conv1D(128, kernel_size=3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        
        # LSTM layers for temporal modeling
        layers.LSTM(128, return_sequences=True),
        layers.Dropout(0.3),
        
        layers.LSTM(64, return_sequences=False),
        layers.Dropout(0.3),
        
        # Dense classification layers
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        
        layers.Dense(len(GESTURES), activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("Model architecture:")
    model.summary()
    
    print("\nTraining CNN-LSTM model...")
    print("This will take 20-40 minutes with GPU...\n")
    
    # Train model
    history = model.fit(
        X_train_w, y_train_cat,
        validation_data=(X_test_w, y_test_cat),
        epochs=50,
        batch_size=32,
        verbose=1
    )
    
    print("\n✅ CNN-LSTM training complete!")
    
    # Plot training history
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy
    ax1.plot(history.history['accuracy'], label='Train')
    ax1.plot(history.history['val_accuracy'], label='Validation')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Loss
    ax2.plot(history.history['loss'], label='Train')
    ax2.plot(history.history['val_loss'], label='Validation')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Evaluate
    test_loss, test_acc = model.evaluate(X_test_w, y_test_cat, verbose=0)
    print(f"\nTest accuracy: {test_acc:.2%}")
    
    # Predictions
    y_pred_probs = model.predict(X_test_w, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test_w, y_pred, target_names=GESTURES))
    
    # Confusion matrix
    cm = confusion_matrix(y_test_w, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=GESTURES, yticklabels=GESTURES)
    plt.title('Confusion Matrix - CNN-LSTM')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
else:
    print("Skipping CNN-LSTM training (SVM selected)")

## 8️⃣ Export Model

Save the trained model to use with the controller.

In [ ]:
import os
from datetime import datetime

# Create export directory
export_dir = "/content/drive/MyDrive/silksong_models/"
os.makedirs(export_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

if MODEL_TYPE == "SVM":
    # Save SVM model and scaler
    model_path = os.path.join(export_dir, f"gesture_classifier_svm_{timestamp}.pkl")
    scaler_path = os.path.join(export_dir, f"feature_scaler_{timestamp}.pkl")
    features_path = os.path.join(export_dir, f"feature_names_{timestamp}.pkl")
    
    joblib.dump(svm_model, model_path)
    joblib.dump(scaler, scaler_path)
    joblib.dump(list(X_df.columns), features_path)
    
    print("✅ SVM model exported!")
    print(f"   Model: {model_path}")
    print(f"   Scaler: {scaler_path}")
    print(f"   Features: {features_path}")
    print("\n📥 Download these files and place them in your project's 'models/' directory")
    
elif MODEL_TYPE == "CNN_LSTM":
    # Save Keras model
    model_path = os.path.join(export_dir, f"gesture_classifier_cnn_lstm_{timestamp}.h5")
    model.save(model_path)
    
    print("✅ CNN-LSTM model exported!")
    print(f"   Model: {model_path}")
    print("\n📥 Download this file and place it in your project's 'models/' directory")

print("\n" + "="*60)
print("🎉 TRAINING COMPLETE!")
print("="*60)
print(f"\nModel type: {MODEL_TYPE}")
print(f"Test accuracy: {test_acc:.2%}" if MODEL_TYPE == "CNN_LSTM" else f"Test accuracy: {svm_model.score(X_test_scaled, y_test):.2%}")
print(f"\nNext steps:")
print("1. Download the model file(s) from Google Drive")
print("2. Place them in your project's 'models/' directory")
print("3. Run the controller: python src/udp_listener.py")

## 📝 Usage Notes

### Using the Model with the Controller

**For SVM:**
```bash
# Place these files in models/:
models/gesture_classifier.pkl
models/feature_scaler.pkl
models/feature_names.pkl

# Run controller
cd src
python udp_listener.py
```

**For CNN-LSTM:**
```bash
# Place this file in models/:
models/cnn_lstm_gesture.h5

# Update udp_listener.py to load CNN-LSTM model instead of SVM
```

### Model Performance Tips

1. **Low accuracy?** Collect more balanced data (30+ samples per gesture)
2. **Certain gestures confused?** Check if they have similar motion patterns
3. **Want better results?** Try CNN-LSTM with GPU for higher accuracy
4. **Training taking too long?** Use SVM for faster results

### Re-training

To re-train with new data:
1. Add new CSV files to the appropriate gesture folders
2. Run this notebook again from the beginning
3. Compare test accuracy before replacing your model

---

**Need help?** Check the project documentation or raise an issue on GitHub.

**Happy gaming! 🎮**